In [1]:
import os

with open(".env") as f:
    for line in f:
        # Ignore comments and empty lines
        if line.strip() and not line.startswith("#"):
            key, value = line.strip().split("=", 1)
            os.environ[key] = value
            
api_key = os.environ.get('API_KEY')
api_key = api_key.strip('"')

In [2]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain
from langchain.tools.arxiv.tool import ArxivQueryRun

In [3]:
import langchain.tools as lt

In [4]:
papers = ArxivQueryRun()
tools = [
    Tool(
        name="papers",
        func=papers.run,
        description="useful for when you need to answer about research paper",
    )
]

In [5]:
prefix = """Have a conversation with a human, answering the following questions as best as you can. You have access to the following tools:"""
suffix = """Begin!

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"]
)

memory = ConversationBufferMemory(memory_key="chat_history")

In [6]:
llm_chain = LLMChain(llm=OpenAI(temperature=0, openai_api_key=api_key), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [7]:
agent_chain.run(input="Tell me about the recent development regarding Retrieval Augmented Generation.")



> Entering new AgentExecutor chain...
Thought: I should look for recent research papers on this topic.
Action: papers
Action Input: Retrieval Augmented Generation
Observation: Published: 2022-02-13
Title: A Survey on Retrieval-Augmented Text Generation
Authors: Huayang Li, Yixuan Su, Deng Cai, Yan Wang, Lemao Liu
Summary: Recently, retrieval-augmented text generation attracted increasing attention
of the computational linguistics community. Compared with conventional
generation models, retrieval-augmented text generation has remarkable
advantages and particularly has achieved state-of-the-art performance in many
NLP tasks. This paper aims to conduct a survey about retrieval-augmented text
generation. It firstly highlights the generic paradigm of retrieval-augmented
generation, and then it reviews notable approaches according to different tasks
including dialogue response generation, machine translation, and other
generation tasks. Finally, it points out some important directions on t

"Recent research papers have highlighted the generic paradigm of retrieval-augmented generation, and have reviewed notable approaches according to different tasks including dialogue response generation, machine translation, and other generation tasks. Hybrid Retrieval-Augmented Generation (HybridRAG) has been proposed to leverage a hybrid setting that combines both client and cloud models, and Augmentation-Adapted Retriever (AAR) has been proposed to learn language model's preferences obtained from a known source language model."